![ACM SIGCHI Summer School on Computational Interaction  
Inference, optimization and modeling for the engineering of interactive systems  
13th June - 18th June 2022  
Saarland University in Saarbrücken, Germany](imgs/header.png)

# Visual clutter classifier

## 🔑  Introduction

### Layout composition 
A website layout is a pattern that defines a website’s structure. It has the role of structuring the information present on a site both for the website’s owner and for users. It provides clear paths for navigation within webpages and puts the most important elements of a website front and center.

There are many reasons to consider a choice of layout wisely, for example:
- A good layout keeps users on the site because it makes important information easily accessible and intuitive to find. A bad layout frustrates users which then quickly leaves the site because they can’t find what they are looking for.

- There’s a strong [relationship](https://blog.hubspot.com/marketing/compelling-stats-website-design-optimization-list) between the layout and the engagement of users with the website. It determines how long they dwell on the website pages, how many pages they browse and how often they come back to the website.

- According to Adobe research, 38% of people will stop engaging with a website if the content/layout is unattractive.

You can think about other reasons why layouts are important. Once you have no doubts, you can start the following exercise.


 Clutter is an important factor in UI
design and information visualization. In this exercise you
will develop a DL model that can predict automatically how visually
packed and/or disorganized a UI is. Overall, cluttered UIs cause
decreased recognition performance, therefore this model has potential
for providing UI designers with "a priori" estimations of visual
clutter, without having to actually look at the UI. This model could be
offered as a third-party service or integrated into an existing one
such as https://interfacemetrics.aalto.fi/.

In [0]:
%tensorflow_version 1.x # this tutorial use TensorFlow v.1

## 1. Import dependencies

In [1]:
import os
from tensorflow.python.keras.models import Sequential #The Sequential model is a linear stack of layers
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.layers import Input, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

# Some GPUs require setting the `allow_growth` setting.
# Comment out this code is you don't have a GPU card.
import tensorflow.compat.v1 as tf
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#tf.Session(config=config)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
!unzip data/layouts-easy.zip #extract data

## 2. Load data

### Dataset description
The data for today's exericise session is a number of layouts which can be categorized into two groups:
- <font color=green>good</font> cop (layout)
- <font color=red>bad</font> cop (layout)

The distinction is based on the composition of the layout.


#### **Bad layout 😭**

![bad](https://media.giphy.com/media/IhgEVrZZ2p8g5g3pFh/giphy.gif)

#### **Good Layout😀**

![good](https://media.giphy.com/media/lSE2LCTTrae4GEy4Km/giphy.gif)

The following code assigns the hyperparameters such as the number of epochs and the batch size. The choice of these hyperparameters affects the accuracy and the training time of the model.

In [5]:
IMAGE_SIZE = (200, 200)
BATCH_SIZE = 64
NUM_EPOCHS = 100

In [6]:
dataset_dir = 'layouts-easy'
trainer_dir = dataset_dir + '/train'

# Specify manually the class labels, otherwise Keras will assign alphanumeric values
# as directories are listed (in no particular order).
#os.walk() yields a 3-tuple (dirpath, dirnames, filenames).
dirname, class_labels, _= next(os.walk(trainer_dir))
print('Class labels: {}'.format(class_labels))

Class labels: ['Bad', 'Good']


###### Data augmentation


Data augmentation is a strategy that enables practitioners to significantly increase the diversity of data available for training models, without actually collecting new data.

Basic augmentation techniques are commonly used to train large neural networks:
- cropping
- padding
- horizontal flipping

More info on data augmentation can be found [here](https://nanonets.com/blog/data-augmentation-how-to-use-deep-learning-when-you-have-limited-data-part-2/).

> Keras ```ImageDataGenerator``` is a gem! It lets you augment your images in real-time while your model is still training! You can apply any random transformations on each training image as it is passed to the model. This will not only make your model robust but will also save up on the overhead memory!

In [7]:
# Apply some data augmentation techniques.
image_data = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
)

*Python Note*🤓

In the next cell we use <font color=green>*generators*</font> - simple functions which return an iterable set of items, one at a time, in a special way.

When an iteration over a set of item starts using the for statement, the generator is run. Once the generator's function code reaches a "yield" statement, the generator yields its execution back to the for loop, returning a new value from the set. The generator function can generate as many values (possibly infinite) as it wants, yielding each one in its turn.

You can write the following code to understand better the logic generator call uses:


```
import random

def lottery():
    # returns 6 numbers between 1 and 40
    for i in range(6):
        yield random.randint(1, 40)

    # returns a 7th number between 1 and 15
    yield random.randint(1,15)

for random_number in lottery():
       print("And the next number is... %d!" %(random_number))
```




In [8]:
# Use generators since we might not be able to fit all the images into memory.
train_generator = image_data.flow_from_directory(
    trainer_dir,
    subset='training',
    color_mode='grayscale',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical', 
    classes=class_labels
)

Found 320 images belonging to 2 classes.


In [9]:
valid_generator = image_data.flow_from_directory(
    trainer_dir,
    subset='validation',
    color_mode='grayscale',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=class_labels
)


Found 80 images belonging to 2 classes.


## 3. Model training

As you will see we use <font color=orange>ReLU</font> activation function in our model. ReLU stands for *rectified linear unit* Mathematically, it is defined as $f(x_i) = max(0, x_i)$. 

Visually, it looks like the following:

![relu](https://miro.medium.com/max/1026/1*DfMRHwxY1gyyDmrIAd-gjQ.png)

ReLU is the most commonly used activation function in neural networks, especially in [CNNs](https://www.youtube.com/watch?v=YRhxdVk_sIs). 

If you are unsure what activation function to use in your network, ReLU is usually a good first choice.

In [10]:
# Notice that images are grayscaled, therefore we have to set the number of channels to 1.
# Image shape is `(width, height, num_channels)` in TensorFlow.
in_shape = IMAGE_SIZE + (1,)

model = Sequential()
model.add(Conv2D(3, (3,3), activation='relu', input_shape=in_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 3)       30        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 3)         0         
_________________________________________________________________
dropout (Dropout)            (None, 99, 99, 3)         0         
_________________________________________________________________
flatten (Flatten)            (None, 29403)             0         
_________________________________________________________________
dense (Dense)                (None, 2)                 58808     
Total params: 58,838
Trainable params: 58,838
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
# Setup some useful callbacks.
#visualizer = TensorBoard(log_dir='/tmp/layouts')
#checkpoint = ModelCheckpoint('{}-best.h5'.format(dataset_dir), monitor='val_acc', mode='max', save_best_only=True)
earlystops = EarlyStopping(patience=10)

In [12]:
# Train the model.
model.fit(
    train_generator,
    epochs=NUM_EPOCHS,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // BATCH_SIZE,
    callbacks=[earlystops]
)

Epoch 1/100
5/5 [==============================] - 10s 2s/step - loss: 12.2280 - accuracy: 0.5250 - val_loss: 10.7759 - val_accuracy: 0.6250
Epoch 2/100
5/5 [==============================] - 8s 2s/step - loss: 8.8444 - accuracy: 0.7031 - val_loss: 12.9213 - val_accuracy: 0.6562
Epoch 3/100
5/5 [==============================] - 8s 2s/step - loss: 9.1573 - accuracy: 0.6938 - val_loss: 4.7931 - val_accuracy: 0.8281
Epoch 4/100
5/5 [==============================] - 8s 2s/step - loss: 6.9396 - accuracy: 0.7844 - val_loss: 7.6810 - val_accuracy: 0.7344
Epoch 5/100
5/5 [==============================] - 8s 2s/step - loss: 7.9143 - accuracy: 0.7563 - val_loss: 14.8970 - val_accuracy: 0.6250
Epoch 6/100
5/5 [==============================] - 8s 2s/step - loss: 9.4503 - accuracy: 0.7719 - val_loss: 6.9813 - val_accuracy: 0.7656
Epoch 7/100
5/5 [==============================] - 8s 2s/step - loss: 5.8745 - accuracy: 0.7906 - val_loss: 8.7825 - val_accuracy: 0.7500
Epoch 8/100
5/5 [============

In [13]:
# Report score from last epoch.
loss, acc = model.evaluate(valid_generator)
print('Accuracy: {:.2f}%'.format(acc*100))

# Finally save the model.
model.save('{}.h5'.format(dataset_dir))

2/2 [==============================] - 1s 208ms/step - loss: 2.9090 - accuracy: 0.9000
Accuracy: 90.00%


As you can see our model showed a result reaching the accuracy of 90%. 

Now, we use our model to make predictions on the test data.

## 4. Model Evaluation

In [14]:
import os
import numpy as np
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

In [12]:
dataset_dir = 'layouts-easy'
trainer_dir = dataset_dir + '/train'

# Specify manually the class labels, otherwise Keras will assign alphanumeric values
# as directories are listed (in no particular order).
dirname, class_labels, _ = next(os.walk(trainer_dir))
print('Class labels: {}'.format(class_labels))



Class labels: ['Bad', 'Good']


In [15]:
# Don't compile the model when testing new data.
model = load_model('{}.h5'.format(dataset_dir), compile=False) #compile the trained model

In [16]:
# Don't shuffle the data, so that we can get the groundtruth labels later.
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        dataset_dir + '/test',
        target_size=(200, 200),
        color_mode='grayscale',
        shuffle = False,
        class_mode='categorical',
        classes=class_labels,
        batch_size=1)

filenames = test_generator.filenames
nb_samples = len(filenames)
predict = model.predict(test_generator,steps = nb_samples)

Found 100 images belonging to 2 classes.


In [17]:
#Let's use the model to predict new data and then compare the predictions against the test labels.

y_true = list(map(lambda f : os.path.dirname(f), test_generator.filenames)) #ground turth
predicted_class_indices=np.argmax(predict,axis=1)
y_pred = list(map(lambda p : class_labels[0] if p == 0 else class_labels[1], predicted_class_indices))

In [20]:
# Let's see how good those predictions were.
precision, recall, fmeasure, _ = precision_recall_fscore_support(y_true, y_pred, labels=class_labels, average='weighted')
print('Precision: {:.2f}%'.format(precision * 100))
print('Recall: {:.2f}%'.format(recall * 100))
print('F-measure: {:.2f}%'.format(fmeasure * 100))

Precision: 82.05%
Recall: 82.00%
F-measure: 81.99%


## 🏁 6. Conclusion

Now, you know:

1.   how important is layout composition
2.   what is data augmentation
3.   what is a generator function in Python
4.   how to apply a simple CNN model
5.   what is a ReLU activation function
